# GPU Configuration and Diagnostics

First, let's run a detailed check of our GPU setup and configuration.

In [ ]:
import torch
import platform
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
import time

def check_gpu_config():
    print("=== System Information ===")
    print(f"Python version: {platform.python_version()}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print("\n=== CUDA Information ===")
        print(f"CUDA version: {torch.version.cuda}")
        print(f"GPU device: {torch.cuda.get_device_name(0)}")
        print(f"Number of GPUs: {torch.cuda.device_count()}")
        
        # Get current GPU memory usage
        print("\n=== GPU Memory Usage ===")
        print(f"Allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
        print(f"Cached: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")
        
        # Run a simple GPU benchmark
        print("\n=== GPU Benchmark ===")
        sizes = [(1000, 1000), (2000, 2000), (4000, 4000)]
        for size in sizes:
            # Clear cache
            torch.cuda.empty_cache()
            
            # Warm-up
            x = torch.randn(size, device='cuda')
            torch.matmul(x, x)
            torch.cuda.synchronize()
            
            # Benchmark
            start_time = time.time()
            x = torch.randn(size, device='cuda')
            y = torch.matmul(x, x)
            torch.cuda.synchronize()
            end_time = time.time()
            
            print(f"Matrix multiplication {size[0]}x{size[1]}: {(end_time - start_time)*1000:.2f} ms")
    else:
        print("\nWARNING: CUDA is not available!")
        print("Please check your GPU drivers and CUDA installation.")

# Run the diagnostics
check_gpu_config()

## Optimized GPU Sequence Analysis

Now let's implement an optimized version of our sequence analysis with better GPU utilization and proper memory management.

In [ ]:
def optimized_seq_to_tensor(seq, device='cuda'):
    """Convert a DNA sequence to one-hot encoded tensor with optimized GPU memory usage"""
    # Pre-allocate memory on GPU
    seq = seq.upper()
    tensor = torch.zeros((len(seq), 4), device=device)
    
    # Vectorized operation for each base
    base_map = {'A': 0, 'T': 1, 'G': 2, 'C': 3}
    for base, idx in base_map.items():
        positions = torch.tensor([i for i, b in enumerate(seq) if b == base], 
                               device=device)
        if len(positions) > 0:
            tensor[positions, idx] = 1
    
    return tensor

def gpu_sequence_similarity_optimized(seq1, seq2, device='cuda', batch_size=1000):
    """Calculate sequence similarity using batched GPU operations"""
    try:
        # Convert sequences to tensors
        with torch.cuda.device(device):
            tensor1 = optimized_seq_to_tensor(seq1, device)
            tensor2 = optimized_seq_to_tensor(seq2, device)
            
            # Use batched processing for large sequences
            similarity = torch.zeros((len(seq1), len(seq2)), device=device)
            for i in range(0, len(seq1), batch_size):
                end_i = min(i + batch_size, len(seq1))
                for j in range(0, len(seq2), batch_size):
                    end_j = min(j + batch_size, len(seq2))
                    
                    # Process batch
                    batch_sim = torch.matmul(
                        tensor1[i:end_i], 
                        tensor2[j:end_j].T
                    )
                    similarity[i:end_i, j:end_j] = batch_sim
                    
                    # Synchronize to ensure completion
                    torch.cuda.synchronize()
            
            return similarity.cpu().numpy()
    
    except torch.cuda.OutOfMemoryError:
        print("GPU OUT OF MEMORY: Trying with smaller batch size...")
        if batch_size > 100:
            return gpu_sequence_similarity_optimized(seq1, seq2, device, batch_size//2)
        else:
            raise

# Test with different sequence lengths
sequence_lengths = [1000, 10000, 100000]
print("Performance comparison at different sequence lengths:")
print("-" * 50)

for length in sequence_lengths:
    test_seq1 = "ATGCTAGCTAGCTGATCG" * (length // 18 + 1)
    test_seq2 = "GCTAGCTAGCTAGCTAGT" * (length // 18 + 1)
    test_seq1 = test_seq1[:length]
    test_seq2 = test_seq2[:length]
    
    print(f"\nTesting with sequence length: {length}")
    
    try:
        # Time CPU version
        torch.cuda.synchronize()  # Ensure GPU is synchronized before timing
        start_time = time.time()
        cpu_result = gpu_sequence_similarity_optimized(test_seq1, test_seq2, device='cpu')
        cpu_time = time.time() - start_time
        
        # Clear GPU cache
        torch.cuda.empty_cache()
        
        # Time GPU version
        torch.cuda.synchronize()
        start_time = time.time()
        gpu_result = gpu_sequence_similarity_optimized(test_seq1, test_seq2, device='cuda')
        torch.cuda.synchronize()
        gpu_time = time.time() - start_time
        
        print(f"CPU time: {cpu_time:.4f} seconds")
        print(f"GPU time: {gpu_time:.4f} seconds")
        print(f"Speedup: {cpu_time/gpu_time:.2f}x")
        
        # Verify results match
        if np.allclose(cpu_result, gpu_result, rtol=1e-4, atol=1e-4):
            print("✓ Results match between CPU and GPU")
        else:
            print("⚠ Warning: Results differ between CPU and GPU!")
            
    except Exception as e:
        print(f"Error at length {length}: {str(e)}")
    
    # Clear GPU memory after each test
    torch.cuda.empty_cache()

# GPU-Accelerated Genetics Analysis

This notebook demonstrates how to leverage GPU acceleration for genomics tasks using PyTorch and CUDA. We'll start by verifying our GPU setup and then implement several GPU-accelerated genomics algorithms.

In [1]:
# Check GPU availability and setup
import torch
import platform

print(f"Python version: {platform.python_version()}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    
    # Test GPU with a simple operation
    x = torch.randn(1000, 1000).cuda()
    y = torch.matmul(x, x)
    print("\nGPU test successful: Matrix multiplication completed")

/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Python version: 3.12.3
PyTorch version: 2.9.0a0+50eac811a6.nv25.09
CUDA available: True
CUDA version: 13.0
GPU device: NVIDIA GeForce RTX 5070 Ti
Number of GPUs: 1

GPU test successful: Matrix multiplication completed


## DNA Sequence Analysis with GPU Acceleration

First, let's create some basic functions for GPU-accelerated DNA sequence analysis. We'll use PyTorch's tensor operations for efficient computation.

In [2]:
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
import time

def seq_to_tensor(seq, device='cuda'):
    """Convert a DNA sequence to one-hot encoded tensor on GPU"""
    # One-hot encoding: A=[1,0,0,0], T=[0,1,0,0], G=[0,0,1,0], C=[0,0,0,1]
    base_to_idx = {'A': 0, 'T': 1, 'G': 2, 'C': 3}
    seq = seq.upper()
    
    # Create tensor on specified device
    one_hot = torch.zeros(len(seq), 4, device=device)
    for i, base in enumerate(seq):
        if base in base_to_idx:
            one_hot[i, base_to_idx[base]] = 1
    return one_hot

def gpu_sequence_similarity(seq1, seq2, device='cuda'):
    """Calculate sequence similarity using GPU acceleration"""
    # Convert sequences to one-hot encoded tensors
    tensor1 = seq_to_tensor(seq1, device)
    tensor2 = seq_to_tensor(seq2, device)
    
    # Calculate similarity score using matrix multiplication
    similarity = torch.matmul(tensor1, tensor2.T)
    return similarity.cpu().numpy()

# Test the functions
test_seq1 = "ATGCTAGCTAGCTGATCG" * 1000  # Make sequence longer for better GPU utilization
test_seq2 = "GCTAGCTAGCTAGCTAGT" * 1000

if torch.cuda.is_available():
    # Time CPU version
    start_time = time.time()
    cpu_result = gpu_sequence_similarity(test_seq1, test_seq2, device='cpu')
    cpu_time = time.time() - start_time
    
    # Time GPU version
    start_time = time.time()
    gpu_result = gpu_sequence_similarity(test_seq1, test_seq2, device='cuda')
    gpu_time = time.time() - start_time
    
    print(f"Sequence Similarity Comparison:")
    print(f"CPU time: {cpu_time:.4f} seconds")
    print(f"GPU time: {gpu_time:.4f} seconds")
    print(f"Speedup: {cpu_time/gpu_time:.2f}x")

Sequence Similarity Comparison:
CPU time: 0.1771 seconds
GPU time: 1.6235 seconds
Speedup: 0.11x


## GPU-Accelerated k-mer Analysis

Next, let's implement k-mer counting using GPU acceleration. This is particularly useful for analyzing large genomic sequences.

In [3]:
import itertools

def generate_kmers(seq, k):
    """Generate all k-mers from a sequence"""
    return [seq[i:i+k] for i in range(len(seq)-k+1)]

def gpu_kmer_frequency(sequences, k=5, device='cuda'):
    """Calculate k-mer frequencies using GPU acceleration"""
    # Generate all possible k-mers
    bases = ['A', 'T', 'G', 'C']
    all_kmers = [''.join(p) for p in itertools.product(bases, repeat=k)]
    kmer_to_idx = {kmer: idx for idx, kmer in enumerate(all_kmers)}
    
    # Initialize frequency tensor on GPU
    freq_tensor = torch.zeros(len(sequences), len(all_kmers), device=device)
    
    # Process each sequence
    for i, seq in enumerate(sequences):
        kmers = generate_kmers(seq, k)
        # Convert k-mers to indices
        indices = torch.tensor([kmer_to_idx[kmer] for kmer in kmers if kmer in kmer_to_idx], 
                             device=device)
        # Count frequencies using GPU operations
        freq_tensor[i].scatter_add_(0, indices, 
                                  torch.ones(len(indices), device=device))
    
    return freq_tensor.cpu().numpy(), all_kmers

# Test with sample sequences
if torch.cuda.is_available():
    test_sequences = [
        "ATGCTAGCTAGCTGATCG" * 1000,
        "GCTAGCTAGCTAGCTAGT" * 1000
    ]
    
    # Time CPU version
    start_time = time.time()
    with torch.device('cpu'):
        cpu_freq, kmers = gpu_kmer_frequency(test_sequences, k=5, device='cpu')
    cpu_time = time.time() - start_time
    
    # Time GPU version
    start_time = time.time()
    gpu_freq, kmers = gpu_kmer_frequency(test_sequences, k=5, device='cuda')
    gpu_time = time.time() - start_time
    
    print(f"K-mer Analysis (k=5):")
    print(f"CPU time: {cpu_time:.4f} seconds")
    print(f"GPU time: {gpu_time:.4f} seconds")
    print(f"Speedup: {cpu_time/gpu_time:.2f}x")
    
    # Show top k-mers for first sequence
    top_kmers = sorted(zip(kmers, gpu_freq[0]), key=lambda x: x[1], reverse=True)[:5]
    print("\nTop 5 k-mers in first sequence:")
    for kmer, freq in top_kmers:
        print(f"{kmer}: {int(freq)}")

K-mer Analysis (k=5):
CPU time: 0.0217 seconds
GPU time: 0.0336 seconds
Speedup: 0.65x

Top 5 k-mers in first sequence:
TAGCT: 2000
GCTAG: 2000
CTAGC: 2000
ATGCT: 1000
AGCTA: 1000


# GPU-Accelerated Genetics Analysis

This notebook demonstrates how to leverage GPU acceleration for genomics tasks using PyTorch and CUDA.

In [4]:
import torch
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
import time

# Helper function to convert DNA sequence to tensor
def seq_to_tensor(seq):
    # One-hot encoding: A=[1,0,0,0], T=[0,1,0,0], G=[0,0,1,0], C=[0,0,0,1]
    base_to_idx = {'A': 0, 'T': 1, 'G': 2, 'C': 3}
    seq_indices = [base_to_idx[base] for base in seq.upper()]
    one_hot = torch.zeros(len(seq), 4)
    one_hot[range(len(seq)), seq_indices] = 1
    return one_hot

# GPU-accelerated sequence similarity scoring
def gpu_sequence_similarity(seq1, seq2, device='cuda'):
    # Convert sequences to one-hot encoded tensors
    tensor1 = seq_to_tensor(seq1).to(device)
    tensor2 = seq_to_tensor(seq2).to(device)
    
    # Calculate similarity score using matrix multiplication
    similarity = torch.matmul(tensor1, tensor2.T)
    return similarity.cpu().numpy()

# Test GPU acceleration
test_seq1 = "ATGCTAGCTAGCTGATCG" * 1000  # Make sequence longer for better GPU utilization
test_seq2 = "GCTAGCTAGCTAGCTAGT" * 1000

# Time CPU version
start_time = time.time()
cpu_result = gpu_sequence_similarity(test_seq1, test_seq2, device='cpu')
cpu_time = time.time() - start_time

# Time GPU version
if torch.cuda.is_available():
    start_time = time.time()
    gpu_result = gpu_sequence_similarity(test_seq1, test_seq2, device='cuda')
    gpu_time = time.time() - start_time
    
    print(f"CPU time: {cpu_time:.4f} seconds")
    print(f"GPU time: {gpu_time:.4f} seconds")
    print(f"Speedup: {cpu_time/gpu_time:.2f}x")
else:
    print("CUDA not available. Please ensure GPU support is properly configured.")

CPU time: 0.1283 seconds
GPU time: 0.5078 seconds
Speedup: 0.25x


## GPU-Accelerated Multiple Sequence Alignment

Here's an example of using GPU acceleration for multiple sequence alignment scoring.

In [5]:
def gpu_multiple_sequence_alignment(sequences, device='cuda'):
    """
    Compute pairwise alignment scores for multiple sequences using GPU acceleration
    """
    n_sequences = len(sequences)
    score_matrix = torch.zeros((n_sequences, n_sequences), device=device)
    
    # Convert all sequences to tensors first
    seq_tensors = [seq_to_tensor(seq).to(device) for seq in sequences]
    
    # Compute all pairwise scores in parallel
    for i in range(n_sequences):
        for j in range(i+1, n_sequences):
            score = torch.sum(torch.matmul(seq_tensors[i], seq_tensors[j].T))
            score_matrix[i,j] = score
            score_matrix[j,i] = score
    
    return score_matrix.cpu().numpy()

# Test with multiple sequences
test_sequences = [
    "ATGCTAGCTAGCTGATCG" * 100,
    "GCTAGCTAGCTAGCTAGT" * 100,
    "TAGCTAGCTAGCTGATCG" * 100,
    "ATGCTAGCTAGCTGATCC" * 100
]

if torch.cuda.is_available():
    # Time CPU version
    start_time = time.time()
    cpu_scores = gpu_multiple_sequence_alignment(test_sequences, device='cpu')
    cpu_time = time.time() - start_time
    
    # Time GPU version
    start_time = time.time()
    gpu_scores = gpu_multiple_sequence_alignment(test_sequences, device='cuda')
    gpu_time = time.time() - start_time
    
    print(f"Multiple Sequence Alignment:")
    print(f"CPU time: {cpu_time:.4f} seconds")
    print(f"GPU time: {gpu_time:.4f} seconds")
    print(f"Speedup: {cpu_time/gpu_time:.2f}x")
    print("\nAlignment Score Matrix:")
    print(gpu_scores)

Multiple Sequence Alignment:
CPU time: 0.0125 seconds
GPU time: 0.0184 seconds
Speedup: 0.68x

Alignment Score Matrix:
[[     0. 820000. 820000. 810000.]
 [820000.      0. 820000. 810000.]
 [820000. 820000.      0. 810000.]
 [810000. 810000. 810000.      0.]]


## GPU-Accelerated k-mer Analysis

Implementing k-mer counting using GPU acceleration.

In [6]:
def generate_kmers(seq, k):
    """Generate all k-mers from a sequence"""
    return [seq[i:i+k] for i in range(len(seq)-k+1)]

def gpu_kmer_frequency(sequences, k=5, device='cuda'):
    """
    Calculate k-mer frequencies using GPU acceleration
    """
    # Generate all possible k-mers
    bases = ['A', 'T', 'G', 'C']
    all_kmers = [''.join(p) for p in itertools.product(bases, repeat=k)]
    kmer_to_idx = {kmer: idx for idx, kmer in enumerate(all_kmers)}
    
    # Initialize frequency tensor on GPU
    freq_tensor = torch.zeros(len(sequences), len(all_kmers), device=device)
    
    # Process each sequence
    for i, seq in enumerate(sequences):
        kmers = generate_kmers(seq, k)
        # Convert k-mers to indices
        indices = torch.tensor([kmer_to_idx[kmer] for kmer in kmers if kmer in kmer_to_idx], 
                             device=device)
        # Count frequencies using GPU operations
        freq_tensor[i].scatter_add_(0, indices, 
                                  torch.ones(len(indices), device=device))
    
    return freq_tensor.cpu().numpy(), all_kmers

# Test with sample sequences
if torch.cuda.is_available():
    import itertools
    
    test_sequences = [
        "ATGCTAGCTAGCTGATCG" * 1000,
        "GCTAGCTAGCTAGCTAGT" * 1000
    ]
    
    # Time CPU version
    start_time = time.time()
    with torch.device('cpu'):
        cpu_freq, kmers = gpu_kmer_frequency(test_sequences, k=5, device='cpu')
    cpu_time = time.time() - start_time
    
    # Time GPU version
    start_time = time.time()
    gpu_freq, kmers = gpu_kmer_frequency(test_sequences, k=5, device='cuda')
    gpu_time = time.time() - start_time
    
    print(f"K-mer Analysis (k=5):")
    print(f"CPU time: {cpu_time:.4f} seconds")
    print(f"GPU time: {gpu_time:.4f} seconds")
    print(f"Speedup: {cpu_time/gpu_time:.2f}x")
    
    # Show top k-mers for first sequence
    top_kmers = sorted(zip(kmers, gpu_freq[0]), key=lambda x: x[1], reverse=True)[:5]
    print("\nTop 5 k-mers in first sequence:")
    for kmer, freq in top_kmers:
        print(f"{kmer}: {int(freq)}")

K-mer Analysis (k=5):
CPU time: 0.0064 seconds
GPU time: 0.0065 seconds
Speedup: 0.98x

Top 5 k-mers in first sequence:
TAGCT: 2000
GCTAG: 2000
CTAGC: 2000
ATGCT: 1000
AGCTA: 1000
